# OPER 610
## Problem 1.6 (Phase 2)
### Brandon Hosley

## Preparation

In [1]:
import pyomo.environ as pyo
model = pyo.AbstractModel()

## Definitions

**Sets**:
$$\begin{align*}
	W &= \{1,2,\ldots,8\}\hspace{19ex} & &:\  \text{Week } w \\
	C &= \{\text{Swiss cheese},\, \text{Sharp cheese}\}\ & &:\  \text{Cheese } c
\end{align*}$$

In [2]:
model.C = pyo.Set()
model.W = pyo.Set(ordered=True)

**Parameters**:

$\tau_{c,w}$ : Entry in the $c$ cheese row, $w$ week column of the original demand table </br>
$\rho_c$ : Lbs of cheese $c$ that an employee can produce per hour </br>
$\omega$ : Hours in the work week </br>
$\alpha$ : Number of employees at the beginning of the period </br>
$\gamma$ : Desired employee growth </br>

$e_w$ : Experienced employees in week $w$ </br>
$i_{c,w}$ : Stored inventory of cheese $c$ at the end of week $w$ </br>
$d_{c,w}$ : Demand of cheese $c$ in week $w$ (Change in units, described below)

In [3]:
model.lbs_hour = pyo.Param(model.C)
model.hours_week = pyo.Param()
model.initial_staff = pyo.Param()
model.desired_growth = pyo.Param()

model.demand = pyo.Param(model.C, model.W)
model.d = pyo.Param(model.C, model.W, mutable=True) 

Early constraint:
$$\begin{align*}
	t,x,d &\geq0 & &\text{(Non-negativity)} \\
\end{align*}$$

In [4]:
model.x = pyo.Var(model.C, model.W, domain=pyo.NonNegativeReals)
model.i = pyo.Var(model.C, model.W, domain=pyo.NonNegativeReals)
model.t = pyo.Var(model.W, domain=pyo.NonNegativeIntegers)
model.e = pyo.Var(model.W, domain=pyo.PositiveIntegers)

## Objective Function

$$\min\quad \sum_{w\in W}e_w$$

In [5]:
def obj_expression(m):
    return sum(m.e[w] for w in m.W)
model.objectivefunction = pyo.Objective(rule = obj_expression)

## Constraints

$$\begin{align*}
\left\lceil\frac{1}{3}t_w \right\rceil + \left\lceil\frac{1}{3}t_{w-1} \right\rceil 
	+ \sum_{c\in C}x_{c,w} &\leq e_w & &\forall\ w\in W & &\text{(Labor availability)} \\
	e_{w-1} + t_{w-2} &= e_w & &\forall w\in W\backslash\{1,2\} & &\text{(Trainees become experienced)} \\
	\sum_{w\in W}t_w &= \gamma && & &\text{(Meeting hiring goal)} \\
	x_{c,w} + i_{c,w-1} - i_{c,w} &= d_{c,w} & &\forall\ c\in C, w\in W & &\text{(Production, inventory, and demand)} \\
	i_{c,w-1} &\leq d_{c,w} & &\forall\ c\in C, w\in W & &\text{(Prevent production of wasted cheese)}
\end{align*}$$

In [6]:
def labor_availability(m,w):
    if w == 1:
        return (m.t[w]/3 + sum(m.x[c,w] for c in m.C)) <= m.e[w] 
    else:
        return (m.t[w]/3 + m.t[w-1]/3 + sum(m.x[c,w] for c in m.C)) <= m.e[w]     
model.LaborConstraint = pyo.Constraint(model.W, rule = labor_availability)

def hiring_goal(m):
    return sum(m.t[w] for w in m.W) == m.desired_growth
model.HiringConstraint = pyo.Constraint(rule = hiring_goal)

def training_completion(m,w):
    if((w == 1) or (w == 2)):
        return m.e[1] == m.e[2]
    else:
        return (m.e[w-1] + m.t[w-2]) == m.e[w]
model.TrainingConstraint = pyo.Constraint(model.W, rule = training_completion)

def cheese_pipeline(m,c,w): 
    if w == 1:
        return (m.x[c,w] - m.i[c,w]) == m.d[c,w]
    else:
        return (m.x[c,w] + m.i[c,w-1] - m.i[c,w]) == m.d[c,w]
model.ProductionConstraint = pyo.Constraint(model.C, model.W, rule = cheese_pipeline)
    
def throttle_cheese(m,c,w):
    return pyo.Constraint.Skip if w == 1 else m.i[c,w-1] <= m.d[c,w]
model.SpoilageConstraint = pyo.Constraint(model.C, model.W, rule = throttle_cheese)

**Unit Conversion**:
$$ d_{c,w} = \frac{1000\tau_{c,w}}{\rho_c \omega} \qquad \forall\ c\in C, w\in W.  $$
**Final Constraints**:
$$\begin{align*}
	e_1 &=\alpha & &\text{(Initial staff)} \\
	t_8 &= 0 & &\text{(Insufficient training time}
\end{align*}$$

In [7]:
inst=model.create_instance('cheese.dat')

for c in inst.C:
    for w in inst.W:
        inst.d[c,w] = ((inst.demand[c,w] * 1000) / (inst.hours_week * inst.lbs_hour[c]))
        
inst.e[1].fix(inst.initial_staff)
inst.t[8].fix(0)
        
inst.pprint()

8 Set Declarations
    C : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'sharp', 'swiss'}
    ProductionConstraint_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    C*W :   16 : {('sharp', 1), ('sharp', 2), ('sharp', 3), ('sharp', 4), ('sharp', 5), ('sharp', 6), ('sharp', 7), ('sharp', 8), ('swiss', 1), ('swiss', 2), ('swiss', 3), ('swiss', 4), ('swiss', 5), ('swiss', 6), ('swiss', 7), ('swiss', 8)}
    SpoilageConstraint_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    C*W :   16 : {('sharp', 1), ('sharp', 2), ('sharp', 3), ('sharp', 4), ('sharp', 5), ('sharp', 6), ('sharp', 7), ('sharp', 8), ('swiss', 1), ('swiss', 2), ('swiss', 3), ('swiss', 4), ('swiss', 5), ('swiss', 6), ('swiss', 7), ('swiss', 8)}
    W : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : 

## Solve

In [11]:
results=pyo.SolverFactory('glpk').solve(inst)

for w in inst.W.value:
    print(' week ', w, ':')
    print('    ', inst.t[w].value, ' new hires.')
    print('    ', inst.e[w].value, ' trained employees.')
    for c in inst.C.value:
        print('        ', c,' :', inst.x[c,w].value)
print('value :', inst.objectivefunction.expr())

 week  1 :
     0.0  new hires.
     60  trained employees.
         sharp  : 37.0833333333333
         swiss  : 22.9166666666667
 week  2 :
     0.0  new hires.
     60.0  trained employees.
         sharp  : 27.9166666666667
         swiss  : 32.0833333333333
 week  3 :
     0.0  new hires.
     60.0  trained employees.
         sharp  : 20.0
         swiss  : 40.0
 week  4 :
     0.0  new hires.
     60.0  trained employees.
         sharp  : 30.0
         swiss  : 30.0
 week  5 :
     5.0  new hires.
     60.0  trained employees.
         sharp  : 32.5
         swiss  : 25.8333333333333
 week  6 :
     0.0  new hires.
     60.0  trained employees.
         sharp  : 30.0
         swiss  : 28.3333333333333
 week  7 :
     25.0  new hires.
     65.0  trained employees.
         sharp  : 15.0
         swiss  : 41.6666666666667
 week  8 :
     0  new hires.
     65.0  trained employees.
         sharp  : 15.0
         swiss  : 41.6666666666667
value : 490.0
